In [ ]:
import numpy as np
import numpy.polynomial as P
import scipy as sp
from matplotlib import pyplot as plt
from tqdm import tqdm
#from sklearn.preprocessing import PolynomialFeatures
from multiprocessing import Pool
import multiprocessing
import ZVnbrosse
from potentials import GaussPotential,GaussMixture,GausMixtureIdent,GausMixtureSame
from samplers import MCMC_sampler,Generate_train,ULA_light
from baselines import set_function,construct_ESVM_kernel,GenerateSigma
from martingale import approx_q,approx_q_independent,test_traj,test_monte_carlo
from optimize import Run_eval_test,optimize_parallel_new 
from utils import *
import copy
import time

In [ ]:
N_burn = 1*10**4 # Burn in period
N_train = 2*10**4 # Number of samples on which we optimize
N_test = 2*10**3 # Number of samples
step = 0.1 # Step size
#step = 0.2
n_traj_train = 10
n_traj_test = 24 # Number of independent MCMC trajectories for test
f_type = "sum_squares"
K_max = 2 #max degree of Hermite polynomial
S_max = 2 #max degree of polynomial during regression stage
lag = 50 #maximal lag order
b_n_train = 20 #lag-window size
b_n_test = int(np.round(N_test**(0.33)))
print(b_n_test)
degree = 2

Choose sampler type (currently only ULA is maintained)

In [ ]:
d = 2
mu = 0.5*np.array([1.0,1.0],dtype = float)
#mu_1 = np.array([-1.0])
#mu_2 = np.array([1.0])
#Sigma_1 = np.array([[1.0]])
#Sigma_2 = np.array([[1.0]])
Sigma = GenerateSigma(d,rand_seed = 777,eps = 0.1) #covariation matrix 
p = 0.5
Cur_pot = GausMixtureSame(Sigma,mu,p)
#Cur_pot = GaussMixture(Sigma_1,Sigma_2,mu_1,mu_2,p)
#Cur_pot = GausMixtureIdent(mu,p)
r_seed = 777
x0 = np.array([0.0,0.0])
fixed_start = True

### Generate data

In [ ]:
r_seed = 777
traj = np.zeros((n_traj_train,N_train,d),dtype = float)
for i in range(n_traj_train):
    cur_traj = ULA_light(r_seed+i,Cur_pot,step, N_burn, N_train, d, return_noise = False, x0 = x0, fixed_start = fixed_start)
    traj[i] = copy.deepcopy(cur_traj)
print(traj.shape)

In [ ]:
inds_arr = np.array([0]) # Taking the second index (not intercept)
params = None    
f_vals = set_function(f_type,traj,inds_arr,params) 
#f_vals = traj[:,:,0]
#f_vals = np.expand_dims(f_vals, axis=2)
print(f_vals.shape)

In [ ]:
n_traj_train_adv = 1*10**5
r_seed = 7771453
traj_adv = np.zeros((n_traj_train_adv,lag,d),dtype = float)

nbcores = multiprocessing.cpu_count()
trav = Pool(nbcores)
res = trav.starmap(ULA_light, [(r_seed+i,Cur_pot,step, 100, lag, d, False,x0, False) for i in range (n_traj_train_adv)])
trav.close()

In [ ]:
print(len(res))
res = np.asarray(res)
print(res.shape)

In [ ]:
for i in range(n_traj_train_adv):
    traj_adv[i] = copy.deepcopy(res[i])
print(traj_adv.shape)

inds_arr = np.array([0]) # Taking the second index (not intercept)
params = None    
f_vals_adv = set_function(f_type,traj_adv,inds_arr,params) 
print(f_vals_adv.shape)

### Evaluate baselines (EVM and ESVM methods)

In [ ]:
sampler = {"sampler":"ULA","burn_type":"full","main_type":"full"} # Sampling method

if sampler["sampler"] == "ULA":
    res = Generate_train(n_traj_train, sampler, Cur_pot, step, N_burn, N_train, d)
    res = np.asarray(res)
    traj_evm,traj_grad = res[:,0,:,:],res[:,1,:,:]
else:
    raise "You should use ULA!"

In [ ]:
print(traj.shape)
inds_arr = np.array([0])#Taking the second index
params = None
f_vals_evm = set_function(f_type,traj_evm,inds_arr,params)

In [ ]:
print(f_vals_evm)
print(f_vals)

In [ ]:
W_train_spec = construct_ESVM_kernel(N_train,b_n_train) #weight matrix for train
W_test_spec = construct_ESVM_kernel(N_test,b_n_test) #weight matrix for test
opt_structure_train = {
    "W":W_train_spec,
    "n_restarts": 3, # Number of restarts during optimization,
    "sigma": 1.0, # Deviation of starting points
    "tol": 1e-5, # Tolerance (for the norm of gradient)
    "alpha": 0.0, # Ridge penalty for 2nd order control functionals
    "beta": 10000.0 # smoothing parameter in the softmax
}
methods = ["ESVM","EVM"]

In [ ]:
coef_dict = optimize_parallel_new(degree,inds_arr,f_vals_evm,traj_evm,traj_grad,opt_structure_train,methods)
print(coef_dict)

In [ ]:
#Create a dictionary and put respective matrices into it
test_params = {
    "W":W_test_spec,
    "step":step,
    "burn_in":N_burn,
    "n_test":N_test,
    "dim":d
}

nbcores = multiprocessing.cpu_count()
trav = Pool(nbcores)
res = trav.starmap(Run_eval_test, [(i,degree,sampler,methods,inds_arr,Cur_pot,test_params,coef_dict,params,f_type) for i in range (n_traj_test)])
trav.close()

In [ ]:
print(len(res))
print(res[0])

In [ ]:
methods_enh = ['Vanilla'] + methods
print(methods_enh)
ints_result = {key: [] for key in methods_enh}
vars_result = {key: [] for key in methods_enh}
time_result = {key: [] for key in methods_enh}

In [ ]:
for i in range(len(res)):
    for j in range(len(methods_enh)):
        ints_result[methods_enh[j]].append(res[i][0][methods_enh[j]][0])
        vars_result[methods_enh[j]].append(res[i][1][methods_enh[j]][0])
        time_result[methods_enh[j]].append(res[i][2][methods_enh[j]][0])
for key in methods_enh:
    ints_result[key] = np.asarray(ints_result[key])
    vars_result[key] = np.asarray(vars_result[key])
    time_result[key] = np.asarray(time_result[key])

### Bernoulli:: Optimize coefficients by solving regression with polynomial features

In [ ]:
#polynomial coefficients
coefs_poly = approx_q(traj,f_vals,n_traj_train,lag,S_max)
print(coefs_poly.shape)

In [ ]:
coefs_poly_adv = approx_q_independent(traj_adv,f_vals_adv,n_traj_train_adv,lag,S_max)
print(coefs_poly_adv.shape)

In [ ]:
ind = 3
print(coefs_poly[ind,:])
print(coefs_poly_adv[ind,:])

In [ ]:
print(np.linalg.norm(coefs_poly_adv-coefs_poly)/np.linalg.norm(coefs_poly))

### Use theoretically computed coefficients in regression

In [ ]:
print(coefs_poly.shape)
print(coefs_poly)
coefs_poly_theor = np.zeros_like(coefs_poly)
for ind in range(len(coefs_poly_theor)):
    if ind == 0:
        coefs_poly_theor[ind,0] = 0
    else:
        coefs_poly_theor[ind,0] = d*(1-(1-step)**(2*ind))/(1-step/2)   
    coefs_poly_theor[ind,3] = (1-step)**(2*ind)
    coefs_poly_theor[ind,5] = (1-step)**(2*ind)

Test our regressors

In [ ]:
cur_lag = 1
N_pts = 100
plt.figure(figsize=(10, 10))
plt.title("Testing regression model",fontsize=20)
plt.plot(traj[0,cur_lag:N_pts],color='r',label='true function')
plt.plot(P.polynomial.polyval(traj[0,:N_pts-cur_lag],coefs_poly[cur_lag,:]),color='g',label = 'practical approximation')
#plt.plot(P.polynomial.polyval(X_train[0,:N_pts-cur_lag],coefs_poly_theor[cur_lag,:]),color='b',label = 'theoretical approximation')
plt.legend(loc = 'lower right',fontsize = 16)
plt.show()

In [ ]:
test_seed = 1453
nbcores = multiprocessing.cpu_count()
trav = Pool(nbcores)
res = trav.starmap(test_traj, [(Cur_pot,coefs_poly_adv,step,test_seed+i,lag,K_max,S_max,N_burn,N_test,d,f_type,inds_arr,params,x0,fixed_start) for i in range (n_traj_test)])
trav.close()

In [ ]:
print(len(res))
print(res[0][0])

In [ ]:
res_new = np.asarray([[res[i][0],res[i][1]] for i in range(len(res))])
print(res_new.shape)

In [ ]:
vars_vanilla = np.var(res_new[:,0,:],axis = 0)
vars_adj = np.var(res_new[:,1,:],axis = 0)
#print(vars_vanilla)
#print(vars_adj)
print(np.mean(vars_adj[1:]/vars_vanilla[1:]))
print(vars_vanilla[-1]/vars_adj[-1])

In [ ]:
print(np.var(ints_result['Vanilla'][:,0])/np.var(ints_result['EVM'][:,0]))
print(np.var(ints_result['Vanilla'][:,0])/np.var(ints_result['ESVM'][:,0]))

### Comparison plots

In [ ]:
title = ""
labels = ['Vanilla\n ULA', 'ULA \nwith MDCV', 'ULA \nwith EVM','ULA\nwith ESVM']

In [ ]:
data = [ints_result['Vanilla'][:,0],res_new[:,1,-1],ints_result['EVM'][:,0],ints_result['ESVM'][:,0]] 
boxplot_ind(data, title, labels)

In [ ]:
title = ""
labels = ['ULA \nwith MDCV', 'ULA \nwith EVM','ULA\nwith ESVM']

In [ ]:
data = [res_new[:,1,-1],ints_result['EVM'][:,0],ints_result['ESVM'][:,0]] 
boxplot_ind(data, title, labels)

In [ ]:
N_burn = 2*10**3
n_traj_train = 24
step = 0.1
N_train_arr = [1*10**3, 1*10**4, 1*10**5, 1*10**6]
Time_arr = np.zeros(len(N_train_arr))
Var_arr = np.zeros(len(N_train_arr))
fixed_start = False
test_seed = 2020

for N in range(len(N_train_arr)):
    cur_func = np.zeros(n_traj_train,dtype = float)
    loc_time = time.time()
    nbcores = multiprocessing.cpu_count()
    trav = Pool(nbcores)
    res = trav.starmap(test_monte_carlo, [(test_seed+j,Cur_pot,step,N_burn,N_train_arr[N],d,False,x0,fixed_start) for j in range(n_traj_train)])
    trav.close()
    cur_traj = np.asarray(res)
    print(cur_traj.shape)
    cur_func = np.mean(np.sum(cur_traj**2,axis=2),axis=1)
    Var_arr[N] = np.var(cur_func)
    Time_arr[N] = (time.time() - loc_time)/4
    print("var = ",Var_arr[N])
    print("time = ",Time_arr[N])

In [ ]:
for N in range(len(Var_arr)):
    print("N = %d, VR rate = %f" % (N_train_arr[N],Var_arr[0]/Var_arr[N]))
    print("N*Var:",N_train_arr[N]*Var_arr[N])